# AgentFacts Verification Demo

This notebook demonstrates the **AgentFacts Registry** - a verifiable metadata system for AI agent governance based on arXiv:2506.13794.

## What You'll Learn
1. Loading agent metadata from synthetic datasets
2. Understanding capabilities with input/output schemas
3. Working with diverse policy types (rate_limit, data_access, approval_required)
4. Registering agents in the registry with bulk operations
5. Verifying agent signatures (tamper detection)
6. Discovering agents by capability, owner, and approval requirements
7. Using `get_capabilities()` and `get_policies()` methods
8. Unregistering agents with audit trail
9. Detecting expired policies
10. Exporting audit data for compliance

## Dataset
We use the synthetic `agent_metadata_10.json` dataset containing 10 diverse agents across 8 teams with rich capabilities and policies.


In [1]:
# Setup imports
import sys
import json
import shutil
from pathlib import Path
from datetime import datetime, timedelta, UTC

sys.path.insert(0, str(Path.cwd().parent))

from backend.explainability.agent_facts import (
    AgentFacts,
    AgentFactsRegistry,
    Capability,
    Policy,
)

# Load synthetic agent metadata from data folder
data_path = Path.cwd().parent / "data" / "agent_metadata_10.json"
with open(data_path) as f:
    agent_data = json.load(f)

print(f"=== Loaded Synthetic Agent Data ===")
print(f"Source: {data_path.name}")
print(f"Total Agents: {len(agent_data)}")
print(f"\nAgents by Owner:")

# Group agents by owner
owners = {}
for agent in agent_data:
    owner = agent["owner"]
    if owner not in owners:
        owners[owner] = []
    owners[owner].append(agent["agent_name"])

for owner, agents in sorted(owners.items()):
    print(f"  {owner}: {', '.join(agents)}")


=== Loaded Synthetic Agent Data ===
Source: agent_metadata_10.json
Total Agents: 10

Agents by Owner:
  analytics-team: Business Report Generator
  customer-success-team: Customer Sentiment Analyzer
  data-engineering-team: Schema Validation Agent
  finance-team: Invoice Data Extractor
  healthcare-ai-team: Medical Diagnosis Assistant
  legal-tech-team: Legal Contract Analyzer
  monitoring-team: Time-Series Anomaly Detector
  personalization-team: Product Recommendation Engine
  research-team: Academic Research Assistant
  security-team: Transaction Fraud Detector


## 1. Exploring Policy Types from Synthetic Data

The synthetic dataset includes three policy types:
- **rate_limit** - Controls API call frequency
- **data_access** - Restricts data field access with encryption requirements  
- **approval_required** - Requires human approval for high-risk operations

Let's examine the diverse policies from our synthetic agents and detect expired ones.


In [2]:
# Analyze policies from synthetic data by type
print("=== Policy Types in Synthetic Data ===\n")

policy_types = {}
all_policies = []

for agent in agent_data:
    for policy in agent.get("policies", []):
        ptype = policy["policy_type"]
        if ptype not in policy_types:
            policy_types[ptype] = []
        policy_types[ptype].append({
            "agent": agent["agent_name"],
            "policy": policy
        })
        all_policies.append((agent["agent_name"], policy))

for ptype, items in sorted(policy_types.items()):
    print(f"📋 {ptype.upper()} ({len(items)} policies)")
    for item in items[:2]:  # Show first 2 examples
        p = item["policy"]
        print(f"   - {p['name']} ({item['agent']})")
        print(f"     Constraints: {list(p['constraints'].keys())}")
    if len(items) > 2:
        print(f"   ... and {len(items) - 2} more")
    print()

# Demonstrate Policy model with approval_required type
print("=== Creating Policies from Synthetic Data ===\n")

# Find diagnosis-generator which has approval_required policy
diagnosis_agent = next(a for a in agent_data if a["agent_id"] == "diagnosis-generator-v1")
approval_policy_data = next(p for p in diagnosis_agent["policies"] if p["policy_type"] == "approval_required")

approval_policy = Policy(**approval_policy_data)
print(f"Approval Required Policy: {approval_policy.name}")
print(f"  Type: {approval_policy.policy_type}")
print(f"  Constraints: {json.dumps(approval_policy.constraints, indent=4)}")
print(f"  Is Effective: {approval_policy.is_effective()}")

# Check for expired policies in synthetic data
print("\n=== Detecting Expired Policies ===\n")

current_time = datetime.now(UTC)
expired_count = 0

for agent_name, policy_data in all_policies:
    policy = Policy(**policy_data)
    if not policy.is_effective():
        expired_count += 1
        print(f"⚠️  EXPIRED: {policy.name}")
        print(f"   Agent: {agent_name}")
        print(f"   Expired: {policy.effective_until}")
        print()

if expired_count == 0:
    print("✓ No expired policies found in synthetic data")
else:
    print(f"Total expired policies: {expired_count}")


=== Policy Types in Synthetic Data ===

📋 APPROVAL_REQUIRED (2 policies)
   - High-Risk Approval (Transaction Fraud Detector)
     Constraints: ['threshold_amount', 'auto_approve_below', 'escalation_timeout_minutes']
   - Physician Approval (Medical Diagnosis Assistant)
     Constraints: ['approval_role', 'max_pending_hours', 'auto_escalate']

📋 DATA_ACCESS (3 policies)
   - Invoice Data Access (Invoice Data Extractor)
     Constraints: ['allowed_fields', 'restricted_fields', 'requires_encryption']
   - HIPAA Compliance (Medical Diagnosis Assistant)
     Constraints: ['allowed_data_sources', 'pii_handling_mode', 'audit_all_access', 'data_retention_days']
   ... and 1 more

📋 RATE_LIMIT (9 policies)
   - API Rate Limit (Invoice Data Extractor)
     Constraints: ['max_calls_per_minute', 'max_calls_per_hour', 'burst_limit']
   - High-Volume Rate Limit (Transaction Fraud Detector)
     Constraints: ['max_calls_per_minute', 'max_calls_per_hour', 'burst_limit']
   ... and 7 more

=== Creatin

## 2. Bulk Registration of Synthetic Agents

The `AgentFactsRegistry` provides persistent storage and management for agent metadata. Let's bulk-register all 10 agents from our synthetic dataset.


In [3]:
# Clean up any existing registry for fresh demo
cache_path = Path.cwd().parent / "cache" / "agent_facts_demo"
if cache_path.exists():
    shutil.rmtree(cache_path)

# Initialize the registry
registry = AgentFactsRegistry(storage_path=cache_path)
print(f"Registry initialized at: {cache_path}")

# Helper function to convert dict to AgentFacts
def create_agent_from_dict(data: dict) -> AgentFacts:
    """Convert synthetic data dict to AgentFacts model."""
    # Convert nested capabilities and policies
    capabilities = [Capability(**c) for c in data.get("capabilities", [])]
    policies = [Policy(**p) for p in data.get("policies", [])]
    
    return AgentFacts(
        agent_id=data["agent_id"],
        agent_name=data["agent_name"],
        owner=data["owner"],
        version=data["version"],
        description=data.get("description", ""),
        capabilities=capabilities,
        policies=policies,
        metadata=data.get("metadata", {}),
    )

# Bulk register all 10 agents from synthetic data
print("\n=== Bulk Registering 10 Agents ===\n")

registered_count = 0
for agent_dict in agent_data:
    agent = create_agent_from_dict(agent_dict)
    registry.register(agent, registered_by=f"{agent.owner}@company.com")
    registered_count += 1
    print(f"✓ Registered: {agent.agent_id}")

print(f"\n=== Registration Summary ===")
print(f"Total agents registered: {registered_count}")

# Show all registered agents grouped by owner
all_agent_ids = registry.list_all()
print(f"\nAgents by owner:")
owner_groups = {}
for agent_id in all_agent_ids:
    agent = registry.get(agent_id)
    if agent.owner not in owner_groups:
        owner_groups[agent.owner] = []
    owner_groups[agent.owner].append(agent)

for owner, agents in sorted(owner_groups.items()):
    print(f"\n  {owner}:")
    for agent in agents:
        caps = len(agent.capabilities)
        pols = len(agent.policies)
        print(f"    - {agent.agent_name} (v{agent.version})")
        print(f"      {caps} capabilities, {pols} policies")
        print(f"      Signature: {agent.signature_hash[:20]}...")


Registry initialized at: /Users/rajnishkhatri/Documents/recipe-chatbot/lesson-17/cache/agent_facts_demo

=== Bulk Registering 10 Agents ===

✓ Registered: invoice-extractor-v2
✓ Registered: fraud-detector-v2
✓ Registered: diagnosis-generator-v1
✓ Registered: contract-reviewer-v1
✓ Registered: research-assistant-v2
✓ Registered: data-validator-v1
✓ Registered: report-generator-v1
✓ Registered: anomaly-detector-v1
✓ Registered: sentiment-analyzer-v1
✓ Registered: recommendation-engine-v1

=== Registration Summary ===
Total agents registered: 10

Agents by owner:

  analytics-team:
    - Business Report Generator (v2.0.7)
      1 capabilities, 1 policies
      Signature: 54f657f82b4c62817ff5...

  customer-success-team:
    - Customer Sentiment Analyzer (v1.0.10)
      1 capabilities, 1 policies
      Signature: ca54d36f5aa0b87d9c81...

  data-engineering-team:
    - Schema Validation Agent (v1.0.6)
      1 capabilities, 1 policies
      Signature: ec97056e734329f55252...

  finance-team:

## 3. Verifying Agent Signatures (Tamper Detection)

Each agent has a cryptographic signature computed from its metadata. If any field is modified without going through the registry, the signature won't match - detecting tampering.


In [4]:
# Verify multiple agents from synthetic data
print("=== Verifying Agent Signatures ===\n")

# Verify all registered agents
verification_results = []
for agent_id in registry.list_all():
    is_valid = registry.verify(agent_id)
    verification_results.append((agent_id, is_valid))

# Show results
valid_count = sum(1 for _, v in verification_results if v)
print(f"Verification Results: {valid_count}/{len(verification_results)} valid\n")

for agent_id, is_valid in verification_results:
    status = "✓ VALID" if is_valid else "✗ INVALID"
    print(f"  {status}: {agent_id}")

# Show detailed signature for one agent
print("\n=== Detailed Signature Analysis ===")
agent = registry.get("fraud-detector-v2")
print(f"\nAgent: {agent.agent_name}")
print(f"Stored Signature:   {agent.signature_hash[:40]}...")
print(f"Computed Signature: {agent.compute_signature()[:40]}...")
print(f"Signatures Match: {agent.signature_hash == agent.compute_signature()}")


=== Verifying Agent Signatures ===

Verification Results: 10/10 valid

  ✓ VALID: invoice-extractor-v2
  ✓ VALID: fraud-detector-v2
  ✓ VALID: diagnosis-generator-v1
  ✓ VALID: contract-reviewer-v1
  ✓ VALID: research-assistant-v2
  ✓ VALID: data-validator-v1
  ✓ VALID: report-generator-v1
  ✓ VALID: anomaly-detector-v1
  ✓ VALID: sentiment-analyzer-v1
  ✓ VALID: recommendation-engine-v1

=== Detailed Signature Analysis ===

Agent: Transaction Fraud Detector
Stored Signature:   322ba193639bce40cc01e49865ff74a5ebb4ae2d...
Computed Signature: 322ba193639bce40cc01e49865ff74a5ebb4ae2d...
Signatures Match: True


In [5]:
# Demonstrate tamper detection using a healthcare agent (high-risk scenario)
print("=== Simulating Tampering on Healthcare Agent ===\n")

# Get the diagnosis generator - a high-risk healthcare agent
original_agent = registry.get("diagnosis-generator-v1")
print(f"Original Agent: {original_agent.agent_name}")
print(f"Original Version: {original_agent.version}")
print(f"Original requires_approval: {original_agent.capabilities[0].requires_approval}")
print(f"Original Signature Valid: {original_agent.verify_signature()}")

# Simulate tampering: remove approval requirement (dangerous!)
tampered_data = original_agent.model_dump()
tampered_data["version"] = "1.4.9-MODIFIED"
# Simulate removing safety controls
tampered_caps = tampered_data["capabilities"]
for cap in tampered_caps:
    cap["requires_approval"] = False  # Remove safety requirement!

# Create agent from tampered data (keeping old signature)
tampered_agent = AgentFacts(**tampered_data)

print(f"\n🚨 TAMPERING DETECTED 🚨")
print(f"Tampered Version: {tampered_agent.version}")
print(f"Tampered requires_approval: {tampered_agent.capabilities[0].requires_approval}")
print(f"\nStored Signature:   {tampered_agent.signature_hash[:40]}...")
print(f"Computed Signature: {tampered_agent.compute_signature()[:40]}...")
print(f"\nTampered Agent Valid: {tampered_agent.verify_signature()}")
print("   ↳ Returns False - unauthorized removal of safety controls detected!")


=== Simulating Tampering on Healthcare Agent ===

Original Agent: Medical Diagnosis Assistant
Original Version: 1.4.9
Original requires_approval: True
Original Signature Valid: True

🚨 TAMPERING DETECTED 🚨
Tampered Version: 1.4.9-MODIFIED
Tampered requires_approval: False

Stored Signature:   a6ba2b753945d2ae4f17956b1c31c33d3e3c23cd...
Computed Signature: f456239fef0da14d5000be40fed8f6eb233256ca...

Tampered Agent Valid: False
   ↳ Returns False - unauthorized removal of safety controls detected!


## 4. Using get_capabilities() and get_policies() Methods

The registry provides dedicated methods for retrieving just capabilities or policies without loading the full agent facts.


In [6]:
# Demonstrate get_capabilities() method
print("=== get_capabilities() Demo ===\n")

# Get capabilities for the fraud detector
capabilities = registry.get_capabilities("fraud-detector-v2")
print(f"Fraud Detector Capabilities ({len(capabilities)}):\n")

for cap in capabilities:
    print(f"  📌 {cap.name}")
    print(f"     Description: {cap.description}")
    print(f"     Latency: {cap.estimated_latency_ms}ms")
    print(f"     Cost: ${cap.cost_per_call}")
    print(f"     Requires Approval: {cap.requires_approval}")
    print(f"     Tags: {cap.tags}")
    print()

# Demonstrate get_policies() method
print("=== get_policies() Demo ===\n")

# Get policies for the diagnosis generator (has approval_required)
policies = registry.get_policies("diagnosis-generator-v1")
print(f"Diagnosis Generator Policies ({len(policies)}):\n")

for policy in policies:
    print(f"  📋 {policy.name} ({policy.policy_type})")
    print(f"     ID: {policy.policy_id}")
    print(f"     Description: {policy.description}")
    print(f"     Effective: {policy.is_effective()}")
    print(f"     Constraints:")
    for k, v in policy.constraints.items():
        print(f"       - {k}: {v}")
    print()

# Find agents with requires_approval capabilities
print("=== Finding High-Risk Agents (requires_approval=True) ===\n")

high_risk_agents = []
for agent_id in registry.list_all():
    caps = registry.get_capabilities(agent_id)
    approval_caps = [c for c in caps if c.requires_approval]
    if approval_caps:
        high_risk_agents.append((agent_id, approval_caps))

print(f"Found {len(high_risk_agents)} agents with approval-required capabilities:\n")
for agent_id, caps in high_risk_agents:
    agent = registry.get(agent_id)
    print(f"  🔒 {agent.agent_name}")
    for cap in caps:
        print(f"     - {cap.name}: {cap.description[:50]}...")


=== get_capabilities() Demo ===

Fraud Detector Capabilities (2):

  📌 score_transaction
     Description: Calculates fraud risk score for a transaction
     Latency: 350ms
     Cost: $0.01
     Requires Approval: False
     Tags: ['fraud', 'ml', 'real-time']

  📌 explain_score
     Description: Provides explanation for fraud score decision
     Latency: 500ms
     Cost: $0.015
     Requires Approval: False
     Tags: ['explainability', 'fraud']

=== get_policies() Demo ===

Diagnosis Generator Policies (2):

  📋 HIPAA Compliance (data_access)
     ID: diagnosis-generator-v1-policy-001
     Description: Ensures HIPAA-compliant data handling
     Effective: False
     Constraints:
       - allowed_data_sources: ['patient_db']
       - pii_handling_mode: redact
       - audit_all_access: True
       - data_retention_days: 365

  📋 Physician Approval (approval_required)
     ID: diagnosis-generator-v1-policy-002
     Description: All diagnoses require physician review
     Effective: True

## 5. Exploring Capability Input/Output Schemas

The synthetic data includes rich JSON schemas defining what each capability expects as input and produces as output.


In [7]:
# Display rich input/output schemas from synthetic data
print("=== Capability Schemas Demo ===\n")

# Show fraud detection score_transaction capability (has complex schema)
fraud_agent = registry.get("fraud-detector-v2")
score_cap = fraud_agent.get_capability("score_transaction")

print(f"Capability: {score_cap.name}")
print(f"Agent: {fraud_agent.agent_name}\n")

print("INPUT SCHEMA:")
print(json.dumps(score_cap.input_schema, indent=2))

print("\nOUTPUT SCHEMA:")
print(json.dumps(score_cap.output_schema, indent=2))

# Show research assistant search_literature capability
print("\n" + "="*60)
research_agent = registry.get("research-assistant-v2")
search_cap = research_agent.get_capability("search_literature")

print(f"\nCapability: {search_cap.name}")
print(f"Agent: {research_agent.agent_name}\n")

print("INPUT SCHEMA:")
print(json.dumps(search_cap.input_schema, indent=2))

print("\nOUTPUT SCHEMA:")
print(json.dumps(search_cap.output_schema, indent=2))

# Compare capabilities across agents by latency and cost
print("\n=== Capability Cost/Performance Comparison ===\n")

all_caps = []
for agent_id in registry.list_all():
    agent = registry.get(agent_id)
    for cap in agent.capabilities:
        all_caps.append({
            "agent": agent.agent_name,
            "capability": cap.name,
            "latency_ms": cap.estimated_latency_ms,
            "cost": cap.cost_per_call or 0,
            "tags": cap.tags,
        })

# Sort by latency
sorted_by_latency = sorted(all_caps, key=lambda x: x["latency_ms"])

print("Fastest capabilities:")
for cap in sorted_by_latency[:5]:
    print(f"  {cap['latency_ms']:>6}ms | ${cap['cost']:.3f} | {cap['capability']} ({cap['agent']})")

print("\nSlowest capabilities:")
for cap in sorted_by_latency[-3:]:
    print(f"  {cap['latency_ms']:>6}ms | ${cap['cost']:.3f} | {cap['capability']} ({cap['agent']})")


=== Capability Schemas Demo ===

Capability: score_transaction
Agent: Transaction Fraud Detector

INPUT SCHEMA:
{
  "type": "object",
  "properties": {
    "transaction_id": {
      "type": "string"
    },
    "amount": {
      "type": "number"
    },
    "merchant": {
      "type": "string"
    },
    "timestamp": {
      "type": "string",
      "format": "date-time"
    },
    "user_id": {
      "type": "string"
    }
  },
  "required": [
    "transaction_id",
    "amount"
  ]
}

OUTPUT SCHEMA:
{
  "type": "object",
  "properties": {
    "fraud_score": {
      "type": "number",
      "minimum": 0,
      "maximum": 1
    },
    "risk_level": {
      "type": "string",
      "enum": [
        "low",
        "medium",
        "high"
      ]
    },
    "flags": {
      "type": "array",
      "items": {
        "type": "string"
      }
    }
  }
}


Capability: search_literature
Agent: Academic Research Assistant

INPUT SCHEMA:
{
  "type": "object",
  "properties": {
    "query": {
      "

## 6. Discovering Agents Across 8 Teams

The registry supports finding agents based on capabilities, ownership, and tags. With 10 agents across 8 teams, we can demonstrate realistic discovery scenarios.


In [8]:
# Find agents by capability across diverse domains
print("=== Finding Agents by Capability ===\n")

# Demonstrate different capability searches
capability_searches = [
    "extract_vendor",      # Financial
    "score_transaction",   # Security/Fraud
    "analyze_symptoms",    # Healthcare
    "assess_risk",         # Legal
    "get_recommendations", # Personalization
    "detect_anomalies",    # Monitoring
]

for cap_name in capability_searches:
    agents = registry.find_by_capability(cap_name)
    if agents:
        agent = agents[0]
        cap = agent.get_capability(cap_name)
        print(f"📍 '{cap_name}':")
        print(f"   Agent: {agent.agent_name} ({agent.owner})")
        print(f"   Latency: {cap.estimated_latency_ms}ms | Cost: ${cap.cost_per_call}")
        print(f"   Tags: {cap.tags}")
        print()

# Find agents by tag patterns
print("=== Finding Agents by Tag Pattern ===\n")

tag_searches = ["fraud", "healthcare", "extraction", "ml"]

for tag in tag_searches:
    matching_agents = []
    for agent_id in registry.list_all():
        agent = registry.get(agent_id)
        for cap in agent.capabilities:
            if tag in cap.tags:
                matching_agents.append((agent.agent_name, cap.name))
                break
    
    if matching_agents:
        print(f"🏷️  Tag '{tag}': {len(matching_agents)} agents")
        for name, cap_name in matching_agents:
            print(f"   - {name} ({cap_name})")


=== Finding Agents by Capability ===

📍 'extract_vendor':
   Agent: Invoice Data Extractor (finance-team)
   Latency: 500ms | Cost: $0.005
   Tags: ['extraction', 'ocr', 'vendor']

📍 'score_transaction':
   Agent: Transaction Fraud Detector (security-team)
   Latency: 350ms | Cost: $0.01
   Tags: ['fraud', 'ml', 'real-time']

📍 'analyze_symptoms':
   Agent: Medical Diagnosis Assistant (healthcare-ai-team)
   Latency: 2000ms | Cost: $0.05
   Tags: ['healthcare', 'diagnosis', 'llm']

📍 'assess_risk':
   Agent: Legal Contract Analyzer (legal-tech-team)
   Latency: 2500ms | Cost: $0.025
   Tags: ['legal', 'risk', 'compliance']

📍 'get_recommendations':
   Agent: Product Recommendation Engine (personalization-team)
   Latency: 50ms | Cost: $0.001
   Tags: ['recommendations', 'personalization', 'ml']

📍 'detect_anomalies':
   Agent: Time-Series Anomaly Detector (monitoring-team)
   Latency: 200ms | Cost: $0.002
   Tags: ['monitoring', 'anomaly', 'time-series']

=== Finding Agents by Tag Patt

In [9]:
# Find agents across all 8 owner teams
print("=== Finding Agents by Owner (All 8 Teams) ===\n")

# Get all unique owners
all_owners = set()
for agent_id in registry.list_all():
    agent = registry.get(agent_id)
    all_owners.add(agent.owner)

print(f"Total teams in registry: {len(all_owners)}\n")

# Show agents for each owner team
for owner in sorted(all_owners):
    agents = registry.find_by_owner(owner)
    print(f"👥 {owner} ({len(agents)} agent{'s' if len(agents) > 1 else ''})")
    for agent in agents:
        caps = [c.name for c in agent.capabilities]
        active_policies = len(agent.get_active_policies())
        print(f"   └─ {agent.agent_name}")
        print(f"      Capabilities: {caps}")
        print(f"      Active Policies: {active_policies}")
        print(f"      Metadata: {agent.metadata.get('deployment_environment', 'N/A')}")
    print()

# Summary statistics
print("=== Registry Summary Statistics ===\n")

total_caps = sum(len(registry.get_capabilities(aid)) for aid in registry.list_all())
total_policies = sum(len(registry.get_policies(aid)) for aid in registry.list_all())
high_risk = sum(1 for aid in registry.list_all() 
                for c in registry.get_capabilities(aid) if c.requires_approval)

print(f"Total Agents: {len(registry.list_all())}")
print(f"Total Teams: {len(all_owners)}")
print(f"Total Capabilities: {total_caps}")
print(f"Total Policies: {total_policies}")
print(f"High-Risk Capabilities (requires_approval): {high_risk}")


=== Finding Agents by Owner (All 8 Teams) ===

Total teams in registry: 10

👥 analytics-team (1 agent)
   └─ Business Report Generator
      Capabilities: ['generate_report']
      Active Policies: 1
      Metadata: production

👥 customer-success-team (1 agent)
   └─ Customer Sentiment Analyzer
      Capabilities: ['analyze_sentiment']
      Active Policies: 1
      Metadata: production

👥 data-engineering-team (1 agent)
   └─ Schema Validation Agent
      Capabilities: ['validate_schema']
      Active Policies: 1
      Metadata: production

👥 finance-team (1 agent)
   └─ Invoice Data Extractor
      Capabilities: ['extract_vendor', 'extract_line_items']
      Active Policies: 2
      Metadata: production

👥 healthcare-ai-team (1 agent)
   └─ Medical Diagnosis Assistant
      Capabilities: ['analyze_symptoms', 'interpret_labs']
      Active Policies: 1
      Metadata: production

👥 legal-tech-team (1 agent)
   └─ Legal Contract Analyzer
      Capabilities: ['extract_clauses', 'assess_r

## 7. Audit Trail Tracking

Every change to agent facts is recorded in an immutable audit trail. This is essential for compliance, debugging, and governance.


In [10]:
# First, let's make some updates to generate audit entries
print("=== Making Updates to Generate Audit Trail ===\n")

# Update the invoice extractor version (from synthetic data)
updated_agent = registry.update(
    agent_id="invoice-extractor-v2",
    updates={
        "version": "1.5.5",
        "description": "Extracts structured data from invoice documents - improved accuracy v2",
    },
    updated_by="developer@finance-team.com"
)
print(f"✓ Updated {updated_agent.agent_id} to version {updated_agent.version}")

# Update the fraud detector with new metadata
updated_fraud = registry.update(
    agent_id="fraud-detector-v2",
    updates={
        "metadata": {
            "deployment_environment": "production",
            "model_version": "gpt-4-turbo",
            "last_health_check": datetime.now(UTC).isoformat(),
        }
    },
    updated_by="ops@security-team.com"
)
print(f"✓ Updated {updated_fraud.agent_id} metadata")

# Update healthcare agent to add a new capability
healthcare_agent = registry.get("diagnosis-generator-v1")
new_cap = Capability(
    name="suggest_treatment",
    description="Suggests treatment options based on diagnosis",
    estimated_latency_ms=2500,
    cost_per_call=0.075,
    requires_approval=True,  # High-risk capability
    tags=["healthcare", "treatment", "llm"],
)
updated_healthcare = registry.update(
    agent_id="diagnosis-generator-v1",
    updates={
        "version": "1.5.0",
        "capabilities": healthcare_agent.capabilities + [new_cap],
    },
    updated_by="physician@healthcare-ai-team.com"
)
print(f"✓ Updated {updated_healthcare.agent_id} with new capability")
print(f"  New capabilities: {[c.name for c in updated_healthcare.capabilities]}")


=== Making Updates to Generate Audit Trail ===

✓ Updated invoice-extractor-v2 to version 1.5.5
✓ Updated fraud-detector-v2 metadata
✓ Updated diagnosis-generator-v1 with new capability
  New capabilities: ['analyze_symptoms', 'interpret_labs', 'suggest_treatment']


In [11]:
# View audit trails for multiple agents
print("=== Audit Trails ===\n")

agents_to_audit = ["invoice-extractor-v2", "fraud-detector-v2", "diagnosis-generator-v1"]

for agent_id in agents_to_audit:
    audit_entries = registry.audit_trail(agent_id)
    agent = registry.get(agent_id)
    print(f"📜 {agent.agent_name} ({len(audit_entries)} entries)")
    print("-" * 50)
    
    for i, entry in enumerate(audit_entries, 1):
        action_icon = {"register": "🆕", "update": "✏️", "verify": "✓", "unregister": "❌"}.get(entry.action, "•")
        print(f"  {action_icon} {entry.action.upper()} by {entry.changed_by}")
        print(f"    Time: {entry.timestamp.strftime('%Y-%m-%d %H:%M:%S UTC')}")
        
        # Show summary of changes (not full content for readability)
        if entry.changes:
            if isinstance(entry.changes, dict):
                change_keys = list(entry.changes.keys())
                if "capabilities" in change_keys:
                    print(f"    Changed: version, capabilities (new capability added)")
                elif "action" in change_keys:
                    print(f"    Action: {entry.changes.get('action', 'N/A')}")
                else:
                    print(f"    Changed: {', '.join(change_keys)}")
        
        if entry.new_signature:
            print(f"    Signature: {entry.new_signature[:16]}...")
        print()
    print()


=== Audit Trails ===

📜 Invoice Data Extractor (3 entries)
--------------------------------------------------
  🆕 REGISTER by finance-team@company.com
    Time: 2025-11-29 19:39:24 UTC
    Action: initial_registration
    Signature: ebd119247489871f...

  ✓ VERIFY by system
    Time: 2025-11-29 19:39:24 UTC
    Changed: result

  ✏️ UPDATE by developer@finance-team.com
    Time: 2025-11-29 19:39:25 UTC
    Changed: version, description
    Signature: a1839486fa28233c...


📜 Transaction Fraud Detector (3 entries)
--------------------------------------------------
  🆕 REGISTER by security-team@company.com
    Time: 2025-11-29 19:39:24 UTC
    Action: initial_registration
    Signature: 322ba193639bce40...

  ✓ VERIFY by system
    Time: 2025-11-29 19:39:24 UTC
    Changed: result

  ✏️ UPDATE by ops@security-team.com
    Time: 2025-11-29 19:39:25 UTC
    Changed: metadata
    Signature: 1caf3e175649e029...


📜 Medical Diagnosis Assistant (3 entries)
--------------------------------------

## 8. Unregistering Agents

Agents can be removed from the registry using the `unregister()` method. This action is recorded in the audit trail for compliance.


In [12]:
# Demonstrate unregister() method
print("=== Unregistering an Agent ===\n")

# Choose an agent to unregister (data-validator is low-risk for demo)
agent_to_remove = "data-validator-v1"
agent_before = registry.get(agent_to_remove)
print(f"Agent to remove: {agent_before.agent_name}")
print(f"  Owner: {agent_before.owner}")
print(f"  Version: {agent_before.version}")
print(f"  Signature: {agent_before.signature_hash[:32]}...")

# Count agents before
count_before = len(registry.list_all())
print(f"\nAgents in registry before: {count_before}")

# Unregister the agent
registry.unregister(agent_to_remove, unregistered_by="admin@company.com")
print(f"\n✗ Unregistered: {agent_to_remove}")

# Verify removal
count_after = len(registry.list_all())
print(f"Agents in registry after: {count_after}")

# Try to get the removed agent (should return None)
removed_agent = registry.get(agent_to_remove)
print(f"Agent retrievable after removal: {removed_agent is not None}")

# Check the audit trail still exists (for compliance)
audit_entries = registry.audit_trail(agent_to_remove)
print(f"\n=== Audit Trail Preserved ===")
print(f"Audit entries for removed agent: {len(audit_entries)}")

# Show the unregister entry
unregister_entry = [e for e in audit_entries if e.action == "unregister"]
if unregister_entry:
    entry = unregister_entry[0]
    print(f"\nUnregister Entry:")
    print(f"  Action: {entry.action.upper()}")
    print(f"  Changed By: {entry.changed_by}")
    print(f"  Previous Signature: {entry.previous_signature[:32]}...")


=== Unregistering an Agent ===

Agent to remove: Schema Validation Agent
  Owner: data-engineering-team
  Version: 1.0.6
  Signature: ec97056e734329f5525275a792871a2c...

Agents in registry before: 10

✗ Unregistered: data-validator-v1
Agents in registry after: 9
Agent retrievable after removal: False

=== Audit Trail Preserved ===
Audit entries for removed agent: 3

Unregister Entry:
  Action: UNREGISTER
  Changed By: admin@company.com
  Previous Signature: ec97056e734329f5525275a792871a2c...


In [13]:
# Export audit data for compliance (using synthetic data agents)
print("=== Exporting Audit Data for Compliance ===\n")

# Export all remaining agents for a comprehensive audit
remaining_agents = registry.list_all()
print(f"Agents to export: {len(remaining_agents)}")

export_path = cache_path / "compliance_export.json"
registry.export_for_audit(
    agent_ids=remaining_agents,
    filepath=export_path
)
print(f"Exported to: {export_path}")

# Show the exported content
with open(export_path) as f:
    export_data = json.load(f)

print(f"\nExport Summary:")
print(f"  Exported At: {export_data['exported_at']}")
print(f"  Agent Count: {export_data['agent_count']}")
print(f"\nAgents in Export:")
for agent_id, data in export_data['agents'].items():
    status = "✓" if data['is_valid'] else "✗"
    print(f"  {status} {agent_id}")
    print(f"    Signature Valid: {data['is_valid']}")
    print(f"    Audit Entries: {len(data['audit_trail'])}")

# Show sample audit trail from export
print("\n=== Sample Audit Trail from Export ===")
sample_agent = "fraud-detector-v2"
if sample_agent in export_data['agents']:
    sample_trail = export_data['agents'][sample_agent]['audit_trail']
    print(f"\n{sample_agent} ({len(sample_trail)} entries):")
    for entry in sample_trail[:3]:  # Show first 3
        print(f"  - {entry['action'].upper()} by {entry['changed_by']}")


=== Exporting Audit Data for Compliance ===

Agents to export: 9
Exported to: /Users/rajnishkhatri/Documents/recipe-chatbot/lesson-17/cache/agent_facts_demo/compliance_export.json

Export Summary:
  Exported At: 2025-11-29T19:39:25.049844+00:00
  Agent Count: 9

Agents in Export:
  ✓ invoice-extractor-v2
    Signature Valid: True
    Audit Entries: 3
  ✓ fraud-detector-v2
    Signature Valid: True
    Audit Entries: 3
  ✓ diagnosis-generator-v1
    Signature Valid: True
    Audit Entries: 3
  ✓ contract-reviewer-v1
    Signature Valid: True
    Audit Entries: 2
  ✓ research-assistant-v2
    Signature Valid: True
    Audit Entries: 2
  ✓ report-generator-v1
    Signature Valid: True
    Audit Entries: 2
  ✓ anomaly-detector-v1
    Signature Valid: True
    Audit Entries: 2
  ✓ sentiment-analyzer-v1
    Signature Valid: True
    Audit Entries: 2
  ✓ recommendation-engine-v1
    Signature Valid: True
    Audit Entries: 2

=== Sample Audit Trail from Export ===

fraud-detector-v2 (3 entrie

## Summary

In this notebook, we demonstrated the complete **AgentFacts Registry** system using synthetic data from `agent_metadata_10.json`:

### API Coverage

| Method | Demonstrated |
|--------|--------------|
| `register()` | Bulk registration of 10 agents |
| `update()` | Version updates, capability additions |
| `verify()` | Signature verification for all agents |
| `get()` | Agent retrieval |
| `get_capabilities()` | Capability-specific queries |
| `get_policies()` | Policy retrieval and filtering |
| `find_by_capability()` | Cross-domain capability search |
| `find_by_owner()` | Team-based discovery (8 teams) |
| `audit_trail()` | Complete change history |
| `export_for_audit()` | Compliance export |
| `list_all()` | Registry enumeration |
| `unregister()` | Agent removal with audit |

### Key Features Demonstrated

1. **Synthetic Data Loading** - 10 diverse agents across 8 teams
2. **Policy Types** - rate_limit, data_access, approval_required
3. **Expired Policy Detection** - Automated policy effectiveness checking
4. **Capability Schemas** - Rich input/output JSON schemas
5. **Tamper Detection** - Cryptographic signature verification
6. **High-Risk Filtering** - Finding `requires_approval` capabilities
7. **Audit Trail Preservation** - History retained even after unregistration

This framework provides the foundation for **explainable and governable AI agents** in enterprise environments.
